## Problem 6-1

(1/1 point)<br>
Suppose you are given a stack of documents and are told that documents with similar sets of keywords are about the same topic. Your job is to organize the documents as best you can by topic. The following 4 questions refer to this situation.

For this situation, it is best to use an unsupervised learning algorithm.<br>
Answer: True

## Problem 6-2

(1/1 point)<br>
Given the above information, which of the following would be the most appropriate feature to use?<br>
Answer: The number of times a particular keyword appears in a document.

## Problem 6-3

(1/1 point)<br>
Your boss comes back with a list of 60 specific keywords as well as 5 specific topics that each keyword is best associated with. Which of the following is true, given this additional information?<br>
Answer: We can use the k-means clustering algorithm with k = 5

## Problem 6-4

(1/1 point)<br>
Your boss comes back one last time with new information. He can now tell you the topic of each document. However, he found some more documents for which the topic is still unknown. Given this information, can we use a supervised learning algorithm to classify the new documents?<br>
Answer: Yes

## Problem 6-5

(1/1 point)<br>
Remember that in Problem Set 6, we used different linkage distance measures to calculate the distances between clusters and decide which cluster a point should belong to. Consider this new method of finding linkage distances, which makes use of the linkage distance methods from the problem set:

```python
def mysteryLinkageDist(self, other):
    av_dist = self.averageLinkageDist(other)
    max_dist = self.maxLinkageDist(other)
    min_dist = self.singleLinkageDist(other)
    retDist = 0.0
    if av_dist == max_dist and max_dist == min_dist:
        retDist = av_dist
    elif av_dist == max_dist:
        retDist = av_dist
    elif av_dist == min_dist:
        retDist = av_dist
    elif max_dist == min_dist:
        retDist = min_dist
    else:
        retDist = random.choice([av_dist,min_dist,max_dist])
    return retDist
```

You are given the following data points with the following feature values:

```
a 4
b 9
c 9
d 9
e 8
f 0
g 8  
```

Answer the following 3 questions based on the above code.

You are asked to run the hierarchical clustering algorithm from Problem Set 6 with the `singleLinkage`, `maxLinkage`, `averageLinkage`, and `mysteryLinkage` distance metrics and asked to report the results at a cutoff of 4 clusters. The final clusters will be the same, no matter which linkage we use.

Answer: True

In [4]:
import pylab, string

def stdDev(X):
    mean = sum(X)/float(len(X))
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    return (tot/len(X))**0.5

def scaleFeatures(vals):
    """Assumes vals is a sequence of numbers"""
    result = pylab.array(vals)
    mean = sum(result)/float(len(result))
    result = result - mean
    sd = stdDev(result)
    result = result/sd
    return result

class Point(object):
    def __init__(self, name, originalAttrs):
        """originalAttrs is an array"""
        self.name = name
        self.attrs = originalAttrs
        
    def dimensionality(self):
        return len(self.attrs)
    
    def getAttrs(self):
        return self.attrs
    
    def distance(self, other):
        #Euclidean distance metric
        result = 0.0
        for i in range(self.dimensionality()):
            result += (self.attrs[i] - other.attrs[i])**2
        return result**0.5
    
    def getName(self):
        return self.name
    
    def toStr(self):
        return self.name + str(self.attrs)
    
    def __str__(self):
        return self.name
    
#City climate example
class City(Point):
    pass

def readCityData(fName, scale = False):
    """Assumes scale is a Boolean.  If True, features are scaled"""
    dataFile = open(fName, 'r')
    numFeatures = 0
    #Process lines at top of file
    for line in dataFile: #Find number of features
        if line[0:4] == '#end': #indicates end of features
            break
        numFeatures += 1
    numFeatures -= 1
    
    #Produce featureVals, cityNames
    featureVals, cityNames = [], []
    for i in range(numFeatures):
        featureVals.append([])
        
    #Continue processing lines in file, starting after comments
    for line in dataFile:
        dataLine = line.strip().split(',') #remove newline; then split
        cityNames.append(dataLine[0])
        for i in range(numFeatures):
            featureVals[i].append(float(dataLine[i+1]))
            
    #Use featureVals to build list containing the feature vectors
    #For each city scale features, if needed
    if scale:
        for i in range(numFeatures):
            featureVals[i] = scaleFeatures(featureVals[i])
    featureVectorList = []
    for city in range(len(cityNames)):
        featureVector = []
        for feature in range(numFeatures):
            featureVector.append(featureVals[feature][city])
        featureVectorList.append(featureVector)
    return cityNames, featureVectorList

def buildCityPoints(fName, scaling):
    cityNames, featureList = readCityData(fName, scaling)
    points = []
    for i in range(len(cityNames)):
        point = City(cityNames[i], pylab.array(featureList[i]))
        points.append(point)
    return points

#Use hierarchical clustering for cities
def hCluster(points, linkage, numClusters, printHistory):
    cS = ClusterSet(City)
    for p in points:
        cS.add(Cluster([p], City))
    history = []
    while cS.numClusters() > numClusters:
        merged = cS.mergeOne(linkage)
        history.append(merged)
    if printHistory:
        print ''
        for i in range(len(history)):
            names1 = []
            for p in history[i][0].members():
                names1.append(p.getName())
            names2 = []
            for p in history[i][1].members():
                names2.append(p.getName())
            print 'Step', i, 'Merged', names1, 'with', names2
            print ''
    print 'Final set of clusters:'
    print cS.toStr()
    return cS


class ClusterSet(object):
    """ A ClusterSet is defined as a list of clusters """
    def __init__(self, pointType):
        """ Initialize an empty set, without any clusters """
        self.members = []
        
    def add(self, c):
        """ Append a cluster to the end of the cluster list
        only if it doesn't already exist. If it is already in the 
        cluster set, raise a ValueError """
        if c in self.members:
            raise ValueError
        self.members.append(c)
        
    def getClusters(self):
        return self.members[:]
    
    def mergeClusters(self, c1, c2):
        """ Assumes clusters c1 and c2 are in self
        Adds to self a cluster containing the union of c1 and c2
        and removes c1 and c2 from self """
        self.add(Cluster(c1.points + c2.points, c1.pointType))
        self.members.remove(c1)
        self.members.remove(c2)        
    
    def findClosest(self, linkage):
        """ Returns a tuple containing the two most similar 
        clusters in self
        Closest defined using the metric linkage """
        bestDist = linkage(self.members[0], self.members[1])
        group = (self.members[0], self.members[1])
        for c1 in self.members:
            for c2 in self.members:
                if c1 == c2:
                    continue
                dist = linkage(c1,c2)
                if dist < bestDist:
                    bestDist = dist
                    group = (c1, c2)
        return group    
    
    def mergeOne(self, linkage):
        """ Merges the two most simililar clusters in self
        Similar defined using the metric linkage
        Returns the clusters that were merged """
        if len(self.members) == 1:
            return None
        if len(self.members) == 2:
            return self.mergeClusters(self.members[0], self.members[1])
        c1, c2 = self.findClosest(linkage)
        self.mergeClusters(c1,c2)
        return (c1, c2)
    
    def numClusters(self):
        return len(self.members)
    
    def toStr(self):
        cNames = []
        for c in self.members:
            cNames.append(c.getNames())
        cNames.sort()
        result = ''
        for i in range(len(cNames)):
            names = ''
            for n in cNames[i]:
                names += n + ', '
            names = names[:-2]
            result += '  C' + str(i) + ':' + names + '\n'
        return result

In [1]:
class Cluster(object):
    """ A Cluster is defined as a set of elements, all having 
    a particular type """
    def __init__(self, points, pointType):
        """ Elements of a cluster are saved in self.points
        and the pointType is also saved """
        self.points = points
        self.pointType = pointType
        
    def singleLinkageDist(self, other):
        """ Returns the float distance between the points that 
        are closest to each other, where one point is from 
        self and the other point is from other. Uses the 
        Euclidean dist between 2 points, defined in Point."""
        dist = float("inf")
        for point in self.points:
            for otherPoint in other.points:
                currentDist = point.distance(otherPoint)
                if currentDist < dist:
                    dist = currentDist
        return dist
    
    def maxLinkageDist(self, other):
        """ Returns the float distance between the points that 
        are farthest from each other, where one point is from 
        self and the other point is from other. Uses the 
        Euclidean dist between 2 points, defined in Point."""
        dist = 0.0
        for point in self.points:
            for otherPoint in other.points:
                currentDist = point.distance(otherPoint)
                if currentDist > dist:
                    dist = currentDist
        return dist
    
    def averageLinkageDist(self, other):
        """ Returns the float average (mean) distance between all 
        pairs of points, where one point is from self and the 
        other point is from other. Uses the Euclidean dist 
        between 2 points, defined in Point."""
        dists = []
        for p1 in self.points:
            for p2 in other.points:
                dists.append(p2.distance(p1))

        if len(dists) > 0:
            return sum(dists) / float(len(dists))
        return 0.0
    
    def mysteryLinkageDist(self, other):
        av_dist = self.averageLinkageDist(other)
        max_dist = self.maxLinkageDist(other)
        min_dist = self.singleLinkageDist(other)
        retDist = 0.0
        if av_dist == max_dist and max_dist == min_dist:
            retDist = av_dist
        elif av_dist == max_dist:
            retDist = av_dist
        elif av_dist == min_dist:
            retDist = av_dist
        elif max_dist == min_dist:
            retDist = min_dist
        else:
            retDist = random.choice([av_dist,min_dist,max_dist])
        return retDist
    
    def members(self):
        for p in self.points:
            yield p
            
    def isIn(self, name):
        """ Returns True is the element named name is in the cluster
        and False otherwise """
        for p in self.points:
            if p.getName() == name:
                return True
        return False
    
    def toStr(self):
        result = ''
        for p in self.points:
            result = result + p.toStr() + ', '
        return result[:-2]
    
    def getNames(self):
        """ For consistency, returns a sorted list of all 
        elements in the cluster """
        names = []
        for p in self.points:
            names.append(p.getName())
        return sorted(names)
    
    def __str__(self):
        names = self.getNames()
        result = ''
        for p in names:
            result = result + p + ', '
        return result[:-2]

In [9]:
points = buildCityPoints('final.txt', False)
hCluster(points, Cluster.maxLinkageDist, 4, False)
hCluster(points, Cluster.averageLinkageDist, 4, False)
hCluster(points, Cluster.singleLinkageDist, 4, False)
hCluster(points, Cluster.mysteryLinkageDist, 4, False)

Final set of clusters:
  C0:a
  C1:b, c, d
  C2:e, g
  C3:f

Final set of clusters:
  C0:a
  C1:b, c, d
  C2:e, g
  C3:f

Final set of clusters:
  C0:a
  C1:b, c, d
  C2:e, g
  C3:f

Final set of clusters:
  C0:a
  C1:b, c, d
  C2:e, g
  C3:f



## Problem 6-6

(1/1 point)<br>
Now you run the hierarchical clustering algorithm from Problem Set 6 with the mysteryLinkage distance metric and look at the results at a cutoff of 3 clusters. The final clusters will be:

Answer: C0:a ||| C1:b,c,d,e,g ||| C2:f

In [10]:
points = buildCityPoints('final.txt', False)
hCluster(points, Cluster.mysteryLinkageDist, 3, False)

Final set of clusters:
  C0:a
  C1:b, c, d, e, g
  C2:f



## Problem 6-7

(1 point possible)<br>
Conceptually, the mysteryLinkage distance metric takes a vote for each data point and decides which cluster to assign the point to based on the majority vote.<br>
Answer: True

In [11]:
points = buildCityPoints('final.txt', False)
hCluster(points, Cluster.maxLinkageDist, 3, False)
hCluster(points, Cluster.averageLinkageDist, 3, False)
hCluster(points, Cluster.singleLinkageDist, 3, False)
hCluster(points, Cluster.mysteryLinkageDist, 3, False)

Final set of clusters:
  C0:a
  C1:b, c, d, e, g
  C2:f

Final set of clusters:
  C0:a
  C1:b, c, d, e, g
  C2:f

Final set of clusters:
  C0:a
  C1:b, c, d, e, g
  C2:f

Final set of clusters:
  C0:a
  C1:b, c, d, e, g
  C2:f

